In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import scipy


In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [4]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(5, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])


In [5]:
# Train the model using fit_generator
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/50


C:\Users\ashut\AppData\Local\Temp\ipykernel_16412\560572671.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(




380/380 [==============================] - 88s 228ms/step - loss: 1.5509 - accuracy: 0.3144 - val_loss: 1.4258 - val_accuracy: 0.4206
Epoch 2/50
380/380 [==============================] - 78s 206ms/step - loss: 1.3655 - accuracy: 0.4357 - val_loss: 1.2867 - val_accuracy: 0.4859
Epoch 3/50
380/380 [==============================] - 76s 200ms/step - loss: 1.2667 - accuracy: 0.4868 - val_loss: 1.2001 - val_accuracy: 0.5284
Epoch 4/50
380/380 [==============================] - 88s 232ms/step - loss: 1.1952 - accuracy: 0.5205 - val_loss: 1.1440 - val_accuracy: 0.5437
Epoch 5/50
380/380 [==============================] - 86s 227ms/step - loss: 1.1418 - accuracy: 0.5441 - val_loss: 1.0924 - val_accuracy: 0.5688
Epoch 6/50
380/380 [==============================] - 85s 223ms/step - loss: 1.0922 - accuracy: 0.5654 - val_loss: 1.0610 - val_accuracy: 0.5809
Epoch 7/50
380/380 [==============================] - 85s 223ms/step - loss: 1.0545 - accuracy: 0.5820 - val_loss: 1.0329 - val_accuracy: 0

In [6]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [7]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')